# Fermi-Hubbard Experiment Example

This notebook demonstrates how to execute a single instance of Fermi-Hubbard experiment on Google processor.

In [ ]:
import cirq
from cirq.contrib.svg import SVGCircuit
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from recirq.fermi_hubbard import (
    ConvertingSampler,
    ExperimentResult,
    FermiHubbardParameters,
    InstanceBundle,
    create_circuits,
    load_experiment,
    plot_quantity,
    quantity_data_frame,
    run_experiment,
    save_experiment
)
from recirq.fermi_hubbard.publication import (
    google_sqrt_iswap_converter,
    ideal_sqrt_iswap_converter,
    parasitic_cphase_compensation,
    rainbow23_layouts,
    trapping_instance
)

# Hide numpy warnings
import warnings
warnings.filterwarnings('ignore')

## Experiment parameters

The first step is to decide on exact experiment parameters like problem Hamiltonian, initial state description as well as mapping of fermions to qubits on the device. All of this information is necessary to create circuits and run the experiment.

### Qubits layout

We're going to use the set of layouts prepared for 23-qubit subgrid of Google Rainbow processor. Multiple layouts are used to average experiment results over different qubits assignments and cancel some of the statistical errors which occur from calibration to calibration.

In [ ]:
layouts = rainbow23_layouts(sites_count=8)

This gives 16 different configurations prepared for 8 sites (16 qubits) each, for example:

In [ ]:
print(layouts[0].text_diagram())

### Problem parameters

Let's use the Hamiltonian with uniform $J=1$ and $U=2$ on each site, initial state prepared as a ground state of a non-interacting Hamiltonian with trapping potential of a Gaussian shape, Trotter step size equal to 0.3, and two particles per chain. The problem parameters with this initial state can be prepared by pre-defined function ```trapping_instance``` (other configurations can be prepared by creating instances of `FermiHubbardPrameters` data class explicitly).

In [ ]:
parameters = [trapping_instance(layout, u=2, dt=0.3, up_particles=2, down_particles=2) for layout in layouts]

The result are instances of `FermiHubbardPrameters` data class for each layout, which uniquely describe configuration to run:

In [ ]:
parameters_example = parameters[0]
parameters_example.hamiltonian

In [ ]:
parameters_example.initial_state

In [ ]:
parameters_example.layout

In [ ]:
parameters_example.dt

## Circuits

This step is not really necesary in order to perform the experiment on a device but it is ilustrative how the Fermi-Hubbard execution works. It shows how to construct circuits that simulate Chemistry problems and run on the Google quantum processor efficiently.

### Circuit creation

The parameters above are just a description of a problem. To create a circuit, a special compilation function is used:

In [ ]:
initial, trotter, measurement = create_circuits(parameters_example, trotter_steps=1)
circuit = initial + trotter + measurement

In [ ]:
circuit

### Circuit decomposition

The circuit above is constructed using gates which are not native to Google hardware, like `cirq.FSim` or `cirq.CZ` with arbitrary exponent. For the purpose of this project a special converter `ConvertToNonUniformSqrtIswapGates` is provided, which supports the primitives required for Fermi-Hubbard problem.

The converter has an ability to decompose gates to $\sqrt{\small \mbox{iSWAP}}$ with unitary parameters deviating from the perfect ones, and varying between qubit pairs. The `google_sqrt_iswap_converter` creates an instance of the converter which approximates to the average values on Rainbow processor (which are approximately equal to `cirq.FSim(π/4, π/24)`).

In [ ]:
google_sqrt_iswap_converter().convert(circuit)

## Cirq Simulation

This section demonstrate how to simulate the above parameters using Cirq simulator. We're going to simulate evolution starting from 0 up to 10 Trotter steps (evolution time between $0$ and $3 \hbar / J$):

In [ ]:
steps = range(11)

### Ideal

To run ideal simulation using the $\sqrt{\small \mbox{iSWAP}}$ gate set, each circuit needs to be converted to $\sqrt{\small \mbox{iSWAP}}$ gate set before execution. The Fermi-Hubbard project provides `ConvertingSampler` that converts circuits before execution. In this case, using the decomposition to `cirq.FSim(π/4, 0)`:

In [ ]:
ideal_sampler = ConvertingSampler(cirq.Simulator(), ideal_sqrt_iswap_converter().convert)

The function `experiment_run` takes the parameters, sampler and list of Trotter steps which describe circuits to compile. It runs them and wraps the outome in `ExperimentResult` data class.

In [ ]:
with tqdm(range(len(parameters) * len(steps))) as progress:
    def post_run(_1, _2):
        progress.update()
    experiments = [run_experiment(params, steps, ideal_sampler, post_run_func=post_run)
                   for params in parameters]

A series of experiments for the same problem instance but with different qubit mappings can be post-processed with the help of `InstanceBundle` class. This class takes care of averaging results over qubits layouts, re-scaling the data by comparing agains reference run (perferct simulation in this case), and extracting various quantities.

In [ ]:
bundle = InstanceBundle(experiments)

In [ ]:
bundle.cache_exact_numerics()

Number of quantities are available, accessible either through ```InstanceBundle``` methods or through dictionary:

In [ ]:
for quantity_name in bundle.quantities:
    print(quantity_name)

They can be converted to Panda's data frame:

In [ ]:
charge_spin_density, _, _ = quantity_data_frame(bundle, 'charge_spin_density')
charge_spin_density

They can also be plotted with ```plot_quantity``` helper function which adjusts appearance according to the data plotted:

In [ ]:
plot_quantity(bundle, 'charge_spin_density');

In [ ]:
plot_quantity(bundle, 'charge_spin_spreading');

### Parasitic controlled-phsae

The Cirq simulator simulates $\sqrt{\small \mbox{iSWAP}}$ gates perfectly. To evaluate the importance of parasitic controlled-phase a circuit could be decomposed to `cirq.FSim(π/4, π/24)` instead of `cirq.FSim(π/4, 0)` and simulated against the latter.

In [ ]:
parasitic_sampler = ConvertingSampler(cirq.Simulator(), google_sqrt_iswap_converter().convert)

with tqdm(range(len(parameters) * len(steps))) as progress:
    def post_run(_1, _2):
        progress.update()
    experiments = [run_experiment(params, steps, parasitic_sampler, post_run_func=post_run) 
                   for params in parameters]

In [ ]:
bundle = InstanceBundle(experiments)
bundle.cache_exact_numerics()

In [ ]:
plot_quantity(bundle, 'charge_spin_density');

In [ ]:
plot_quantity(bundle, 'charge_spin_spreading', show_std_error=True);

# Google's Quantum Computing Service Execution

In order to run experiment on Google's QCS, a special sampler that runs circuits at Google's QCS is necessary. The script below assumes that an environment variable ```GOOGLE_CLOUD_PROJECT``` is present and set to a valid Google Cloud Platform project identifier.

In [ ]:
processor_id = 'rainbow'
engine_sampler = cirq.google.get_engine_sampler(processor_id=processor_id,
                                                gate_set_name='sqrt_iswap')
google_sampler = ConvertingSampler(engine_sampler, google_sqrt_iswap_converter().convert)

It is convenient to watch for a progress of the QCS execution. It is also good to persist the experiment results on disk as soon sa they are ready. Although rare, the remote operation might fail for various reasons and more advanced execution workflow might include error handling, experiment pause and continuation, etc.

The script below does not include Floquet calibration.

In [ ]:
results_dir = 'trapping'

with tqdm(range(len(layouts) * len(steps))) as progress:
    def post_run(_1, _2):
        progress.update()
    for index, params in enumerate(parameters):
        experiment = run_experiment(params, steps, google_sampler, post_run_func=post_run)
        save_experiment(experiment, f'{results_dir}/trapping_{index + 1}.json')

In [ ]:
experiments = [load_experiment(f'{results_dir}/trapping_{index + 1}.json') 
               for index in range(len(parameters))]

In [ ]:
bundle = InstanceBundle(experiments=experiments,
                        numerics_transform=parasitic_cphase_compensation(0.138))
bundle.cache_exact_numerics()

In [ ]:
plot_quantity(bundle, 'charge_spin_density', show_std_error=True);

In [ ]:
plot_quantity(bundle, 'charge_spin_spreading', show_std_error=True);